In [ ]:
import os 
import sys
import torch
import ast
import trimesh
from tqdm import tqdm
import open3d as o3d
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy.spatial.transform import Rotation 

notebook_path = os.path.abspath("../")
os.chdir(notebook_path)
from mvtorch.view_selector import MVTN

In [ ]:
# Get the absolute path of MYPOINTCLIP (two levels up)
root_dir = os.path.abspath("/home/mpelissi/CLIP/myCLIP2Point/")
from functions_pov_clip2pt import *
print("Root directory:", root_dir)

from render.selector import Selector
from render.render import *
from utils_clip2point import read_state_dict, read_ply, create_rotation_matrix, pcshow, write_obj_with_color, calculer_aires_triangles_batch, save_colored_obj_with_faces, read_paths_from_txt

# Model 3D remeshing iso
dir_remeshing =  "/home/mpelissi/Dataset/ModelNet40_remeshing_iso"
dir_output = "/home/mpelissi/MVTN/my_MVTN/outputs"

In [ ]:
# Global parameters
nb_views = 12; views_config = 'circular'
points_per_pixel=1; points_radius=0.02; image_size=224

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu');  print("💻 device : ", device)
mvtn = MVTN(nb_views, views_config).cuda()

# Pour 1 obj


In [ ]:
path_ply = "10_03.ply"
path_obj = correspondence_df[correspondence_df['path_ply'] == path_ply]['path_obj'].values[0]
print("ply:", path_ply)
print("obj:", path_obj)
cat = path_obj.split('/')[-3]; type = path_obj.split('/')[-2]; name = path_obj.split('/')[-1].split('_SM')[0]
print(cat, type, name)

In [ ]:
path_obj.split('/')[-1]

Alignement OBJ et PLY

In [ ]:
## Récupération des coordonnées des sommets, des faces et des normales
mesh_init = trimesh.load_mesh(os.path.join(dir_remeshing, path_obj+".obj"))
# Save point cloud to .ply file
view_pcd_init = o3d.geometry.PointCloud(); view_pcd_init.points = o3d.utility.Vector3dVector(np.array(mesh_init.vertices))
# Save the point cloud to a .ply file
o3d.io.write_point_cloud(f"{os.path.join(dir_output, name)}_myview_init_v2.ply", view_pcd_init)
## Rotation pour aligner avec les PLY-limper
rotation_to_apply = ast.literal_eval(rotations[rotations['path_ply'] == path_ply]['transformation'].values[0])

mesh_aligned = mesh_init.copy()
for transf in rotation_to_apply:
    angle = transf[1]
    matrix = create_rotation_matrix('Z', angle)    
    mesh_aligned.apply_transform(matrix)
    
## Récupération des coordonnées des sommets, des faces et des normales
array_coords_aligned = np.array(mesh_aligned.vertices); nb_vertices = len(array_coords_aligned)
print("nb vertices aligned : ", nb_vertices)
array_normals_aligned = np.array(mesh_aligned.vertex_normals)
array_faces = np.array(mesh_aligned.faces); nb_faces = len(array_faces)
print("nb faces aligned : ", nb_faces)

## Save point cloud to .ply file
view_pcd_aligned = o3d.geometry.PointCloud()
view_pcd_aligned.points = o3d.utility.Vector3dVector(array_coords_aligned)
view_pcd_aligned.normals = o3d.utility.Vector3dVector(array_normals_aligned)
# Save the point cloud to a .ply file
output_file = f"{os.path.join(dir_output, name)}_myview_init_aligned_v2-normals.ply"
o3d.io.write_point_cloud(output_file, view_pcd_aligned)

In [ ]:
a

In [ ]:
## Convert to a NumPy array and take only XYZ coordinates
points_aligned = torch.tensor(array_coords_aligned, dtype=torch.float32).cuda()
normals_aligned = torch.tensor(array_normals_aligned, dtype=torch.float32).cuda()
print("points_aligned shape:", points_aligned.shape)  # Expected: [N, 3]
print("normals_aligned shape:", normals_aligned.shape)  # Expected: [N, 3]

# Ensure the point cloud has the correct shape [B, N, 3]
B = 1  # Batch size
points_aligned = points_aligned.unsqueeze(0)  # Add batch dimension -> [1, N, 3]
normals_aligned = normals_aligned.unsqueeze(0)  # Add batch dimension -> [1, N, 3]
print("\npoints_aligned shape:", points_aligned.shape)  # Expected: [1, N, 3]
print("normals_aligned shape:", points_aligned.shape)  # Expected: [1, N, 3]

Projections dans les 6 vues

In [ ]:
# Cameras positions
c_views_azim, c_views_elev, c_views_dist = selector(points_aligned)
print(points_aligned.shape[0])
print(c_views_azim.shape, c_views_elev.shape, c_views_dist.shape)  # Expected: [1, 12]
print(c_views_elev)

In [ ]:
# Depthmaps
fragments, world_points, cameras, world_mesh, world_normals, Rs, Ts = my_render_mesh(points_aligned, normals_aligned, array_faces, c_views_azim, c_views_elev, c_views_dist, views, image_size, device)
# transform xyz to the camera view coordinates
cam_points = cameras.get_world_to_view_transform().transform_points(world_points)
cam_points_np = cam_points.cpu().numpy()
# transform xyz to the camera view coordinates
cam_normals = cameras.get_world_to_view_transform().transform_normals(world_normals)
cam_normals_np = cam_normals.cpu().numpy()
# Cartes de profondeur
depthmaps_np = fragments.zbuf[:,:,:,0].cpu().numpy()

In [ ]:
# Assuming depthmap_i is generated for each view in the loop from cell 13
fig, axes = plt.subplots(views//6, 6, figsize=(20, 6))
fig.suptitle("My Depth Images "+name, fontsize=16)

for i in range(views):
    row, col = divmod(i, 6)    
    axes[row, col].imshow(depthmaps_np[i], cmap='viridis')
    axes[row, col].set_title(f"View {i+1} {c_views_azim[0,i]} {c_views_elev[0,i]} {c_views_dist[0,i]}")
    cbar = fig.colorbar(plt.cm.ScalarMappable(cmap='viridis'), ax=axes[row, col], orientation='vertical', fraction=0.046, pad=0.04)
    cbar.set_label('Depth Value')

plt.tight_layout()
plt.show()

In [ ]:
# pix_to_face is of shape (N, H, W, 1)
pix_to_face = fragments.pix_to_face ; print("pix_to_face shape:", pix_to_face.shape)  # Expected: [N, H, W, 1]
pix_to_face = fragments.pix_to_face[..., 0] ; print("pix_to_face shape:", pix_to_face.shape)  # Expected: [N, H, W]
print("pix_to_face shape:", pix_to_face.shape)  # Expected: [N, H, W]

In [ ]:
visible_faces_per_view = []
visible_verts_per_view = []
#faces = world_mesh.faces_packed()  # (F, 3)

for b in tqdm(range(pix_to_face.shape[0])):
    array_pt_cloud_b = cam_points_np[b]
    array_normals_b = cam_normals_np[b]
    
    # 1. Get valid face indices (ignore -1 = background)
    face_ids = pix_to_face[b]
    idx_visible_faces_b = torch.unique(face_ids[face_ids >= 0]) % nb_faces  # remove -1 (background)
    idx_visible_faces_b_np = idx_visible_faces_b.cpu().numpy()
    visible_faces_per_view.append(idx_visible_faces_b)
    print(f"Visible faces for view {b}: {idx_visible_faces_b.shape}")  
    
    # 2. Extract visible vertices from those faces
    idx_visible_verts_b = torch.unique(torch.tensor(array_faces)[idx_visible_faces_b])
    idx_visible_verts_b_np = idx_visible_verts_b.cpu().numpy()
    visible_verts_per_view.append(idx_visible_verts_b)   
    print(f"Visible vertices for view {b}: {idx_visible_verts_b.shape}") 
    
    obj_filename = f"{os.path.join(dir_output, name)}_myview_{b+1}_color.obj"
    write_obj_with_color(array_pt_cloud_b, array_faces, idx_visible_verts_b_np, obj_filename)
    
    ## Angle sommets visibles
    cos_angle_visible_vertex_b = np.full(array_pt_cloud_b.shape[0], np.nan)
    # sommets et normales visibles
    v_visible = array_pt_cloud_b[idx_visible_verts_b_np]
    n_visible = array_normals_b[idx_visible_verts_b_np]
    n_norms = np.linalg.norm(n_visible, axis=1, keepdims=True)
    n_norms[np.where(n_norms == 0)] = 1e-10 # quelques normales sont nulles
    # normalisation
    n_visible_norm = n_visible / n_norms
    # vecteurs directeurs
    D = -v_visible
    D /= np.linalg.norm(D, axis=1, keepdims=True)
    cos_alpha = np.abs(np.sum(D * n_visible_norm, axis=1))
    cos_angle_visible_vertex_b[idx_visible_verts_b_np] = cos_alpha   
    obj_filename_angle = f"{os.path.join(dir_output, name)}_myview_{b+1}_angles.obj"
    save_colored_obj_with_faces(obj_filename_angle, array_pt_cloud_b, cos_angle_visible_vertex_b, array_faces)
    
    
    # Surface Totale 3D
    surface3D_b = np.sum(calculer_aires_triangles_batch(array_pt_cloud_b, array_faces))
    # Surface visible de la projection courante
    surface3D_visible_b = np.sum(calculer_aires_triangles_batch(array_pt_cloud_b, array_faces[idx_visible_faces_b_np]))
    
    #data_output_path = os.path.join(dir_output, cat, type, name+"_cam"+str(i+1)+"_data.npz")
    data_output_path = os.path.join(dir_output, name+"_cam"+str(b+1)+"_data.npz")

    # Step 1: Save all array data in a compressed .npz file
    # np.savez_compressed(
    #     data_output_path,
    #     array_pt_cloud = array_pt_cloud_b,
    #     array_normals = array_normal_b,
    #     dephtmap = depthmaps_np[b],
    #     visible_vertex_idx = idx_visible_verts_b_np,
    #     visible_faces = idx_visible_faces_b_np,
    #     cos_angles = cos_angle_visible_vertex_b,
    #     surface3D = surface3D_b,
    #     surface3D_visible = surface3D_visible_b)

In [ ]:
np.max(idx_visible_faces_b_np)